# Input file

In [17]:
with open('data.txt', 'r') as f:
    data = f.read()

In [18]:
chars = sorted(list(set(data)))
vocab_size = len(chars)

# Bigram Model

In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(20230420)

class MyBigramModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, idx, targets=None):
        # idx is (B, T) array of indices in the current context
        # targets is (B, T) array of indices in the next context
        # get the embeddings
        embeddings = self.embedding(idx) # (B, T, E)
        # run the RNN
        rnn_out, _ = self.rnn(embeddings) # (B, T, H)
        # get the logits
        logits = self.linear(rnn_out) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_len=100):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_len):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [115]:
embedding_dim = 10
hidden_dim = 20

itos = { i:ch for i,ch in enumerate(chars) }
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

model = MyBigramModel(vocab_size, embedding_dim, hidden_dim)

x = torch.randint(0, vocab_size, (3, 2))
y = torch.randint(0, vocab_size, (3, 2))

out, loss = model(x, y)
print(out.shape, loss)

x = torch.randint(0, vocab_size, (3, 1))
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_len=100)[0].tolist()))


torch.Size([6, 65]) tensor(4.3064, grad_fn=<NllLossBackward0>)

EgfLyxQz.waDtSt;e'FIRMdJ;sCfnDSpC,;DMGEIle-q?rWTzDtQhEDKioaH!cP$cqra'G d,RPRGTtxrs
?;kGBb?q&geSimb,L
